# Предсказание для test.csv

In [1]:
import pandas as pd
import numpy as np
import pickle

In [8]:
test_data = pd.read_csv('./data/test.csv', index_col='id')
features = pd.read_csv('./data/features.csv')
test_data.head(5)

,lat,lon
id,,
3084,55.782276,49.148234
3085,56.841500,35.853059
3086,55.356780,86.156697
3087,56.755087,60.703962
3088,59.907438,30.250297


In [4]:
features.head()

,lat,lon,0,1,2,3,4,5,6,7,...,353,354,355,356,357,358,359,360,361,362
0,59.837100,30.262701,0.411159,0.360032,0.350883,0.476389,0.373542,0.399907,0.397579,0.486442,...,0.041906,0.045916,0.085368,0.093044,0.014491,0.004091,0.043260,0.045916,0.060025,0.071592
1,59.935705,30.274201,0.500628,0.458489,0.522933,0.538892,0.482807,0.497257,0.541007,0.482956,...,0.041262,0.054171,0.066940,0.057510,0.007990,0.017897,0.042617,0.054171,0.047068,0.044251
2,59.929714,30.419863,0.450037,0.413593,0.445300,0.490658,0.403224,0.448099,0.516285,0.526028,...,0.044056,0.052290,0.126482,0.116896,0.006771,0.007329,0.045406,0.052290,0.088934,0.089945
3,56.111227,40.356119,0.160964,0.146153,0.301632,0.196871,0.157927,0.130723,0.246422,0.385300,...,0.043072,0.051181,0.042962,0.030555,0.010293,0.002216,0.044425,0.051181,0.030208,0.023510
4,56.233351,43.872728,0.102522,0.073835,0.270109,0.129343,0.106780,0.082463,0.201381,0.372377,...,0.047839,0.048137,0.135698,0.163684,0.009615,0.003579,0.049185,0.048137,0.095414,0.125946


In [14]:
from sklearn.feature_selection import VarianceThreshold

def correlation_delete(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] >= threshold and corr_matrix.columns[j] not in col_corr:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname]

def transform_data(dataset):
    cutter = VarianceThreshold(threshold=0.05)
    cutter.fit(dataset)
    constant_cols = [x for x in dataset.columns if x not in np.delete(cutter.get_feature_names_out(), [0, 1])]
    
    dataset = dataset[constant_cols]

    correlation_delete(dataset, 0.55)

    step = 1.5
    dataset['lat_bin'] = (dataset['lat'] // step) * step
    dataset['lon_bin'] = (dataset['lon'] // step) * step
    
    grouped_features_df = dataset.groupby(['lat_bin', 'lon_bin']).mean().reset_index()
    
    test_data['lat_bin'] = (test_data['lat'] // step) * step
    test_data['lon_bin'] = (test_data['lon'] // step) * step
    
    merged_df = pd.merge(test_data, grouped_features_df, on=['lat_bin', 'lon_bin'], how='left')
    merged_df.drop(['lat_bin', 'lon_bin'], axis=1, inplace=True)

    merged_df = merged_df.drop(['lat_y', 'lon_y'], axis=1)
    merged_df['id'] = range(3084, 4113)
    merged_df = merged_df.set_index('id')

    return merged_df

In [15]:
model = pickle.load(open('./model/model.sav', 'rb'))

In [16]:
prep_data = transform_data(features)

C:\Users\Vlad\AppData\Local\Temp\ipykernel_2152\2521517999.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['lat_bin'] = (dataset['lat'] // step) * step
C:\Users\Vlad\AppData\Local\Temp\ipykernel_2152\2521517999.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['lon_bin'] = (dataset['lon'] // step) * step


In [20]:
data_to_export = {
    'id': range(3084, 4113),
    'score': model.predict(prep_data)
}

In [23]:
submission = pd.DataFrame(data=data_to_export)
submission = submission.set_index('id')

In [25]:
submission.to_csv('submission.csv')